In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import csv
import tensorflow as tf
import nltk 

import gensim as gs

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.contrib import rnn

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

nltk.download('punkt')

df1 = pd.read_csv('HillaryTrumpTrainWithTags.csv')
df2 = pd.read_csv('TrumpHillaryTest.csv')
"""
df2 = pd.read_table('glove.twitter.27B.25d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

def vectorize(w):
  return df2.loc[w].as_matrix()
"""
  

[nltk_data] Downloading package punkt to /home/karthik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'\ndf2 = pd.read_table(\'glove.twitter.27B.25d.txt\', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)\n\ndef vectorize(w):\n  return df2.loc[w].as_matrix()\n'

In [2]:
def handleToNumber(handle):
    if handle == "realDonaldTrump":
        return 0
    else:
        return 1

In [3]:
df1['tokenized_tweets'] = df1.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)
df1['labels'] = df1.apply(lambda row: handleToNumber(row['handle']), axis=1)
df2['tokenized_tweets'] = df2.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

In [4]:
tweets = df1['tokenized_tweets']
tweets_test = df2['tokenized_tweets']

In [5]:
length = []
_ = df1.apply(lambda row: length.append(len(row['tokenized_tweets'])), axis=1)
max_length = max(length)

In [6]:
max_length

54

In [7]:
vectorizer = gs.models.Word2Vec(df1['tokenized_tweets'].append(df2['tokenized_tweets']) , min_count=1, size = 25, iter = 10)

In [8]:
train_matrix = []

for i, tweet in enumerate(tweets):
    #print(type(string))
    train_matrix.append(vectorizer.wv[tweet])

In [9]:
# np.array(train_matrix).reshape(len(train_matrix),1)

In [10]:
train_matrix = np.array(train_matrix)
train_matrix[0]

array([[-8.06081116e-01, -2.19484344e-01, -3.88274312e-01,
        -1.61107138e-01, -1.00259507e+00,  3.27443928e-01,
        -1.11286491e-01, -6.41066313e-01,  1.22273910e+00,
         1.51146317e+00,  5.44606924e-01,  8.63679528e-01,
         1.99143612e+00,  6.36692703e-01,  6.16807878e-01,
         2.18709469e-01,  2.33206689e-01, -6.15914166e-01,
         9.98988688e-01, -7.40315318e-01, -1.08086300e+00,
         5.99119104e-02,  1.49287045e+00,  7.63807058e-01,
        -8.84026170e-01],
       [-2.07941487e-01,  2.03470401e-02,  2.27209833e-02,
         2.17259526e-01, -3.01008165e-01,  1.78454652e-01,
        -1.20773606e-01, -2.59526819e-02,  4.34588730e-01,
         2.96456426e-01,  4.70216304e-01, -7.75579438e-02,
         5.98252773e-01, -1.21941613e-02,  2.69649953e-01,
        -1.87879547e-01, -1.22828498e-01, -1.45573393e-01,
         6.30454421e-01, -2.89097577e-01, -5.18052578e-01,
         4.21677500e-01,  5.24195910e-01,  2.36306012e-01,
        -4.38531160e-01],
    

In [11]:
type(train_matrix)

numpy.ndarray

In [12]:
train_matrix.shape

(5000,)

In [13]:
train_matrix[0].shape

(23, 25)

In [14]:
twit = train_matrix[4999]

In [15]:
def pad(twits):
    """
    Does the padding as many times as required
    """
    l = len(twits)
    d= twits.shape[1]
    padding = np.zeros(d)
    
    for i in range(0,max_length-l):
        twits = np.append(twits,padding)
        
    return twits.reshape(max_length,d)

In [16]:
def pad_matrix(t_matrix):
    """
    Loops through all the vectorized tweets and calls the pad function on each of them.
    """
    
    padded_matrix =[]
    for i,vectorized_tweet in enumerate(t_matrix):
         padded_matrix.append(pad(vectorized_tweet))
    padded_matrix = np.array(padded_matrix)
    
    return padded_matrix

In [17]:
padded_train_matrix = pad_matrix(train_matrix)

In [18]:
padded_train_matrix.shape

(5000, 54, 25)

In [19]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [20]:
labels = df1['labels']
labels_vector = []

for i, label in enumerate(labels):
    labels_vector.append([label, 1 - label])
labels_vector = np.array(labels_vector)

In [21]:
#define constants
#unrolled through 28 time steps
time_steps=54
#hidden LSTM units
num_units=128
#rows of 28 pixels
n_input=25
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=2
#size of batch
batch_size=128

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

#loss_function
loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<800:
        batch_x,batch_y=next_batch(batch_size, padded_train_matrix, labels_vector)
        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

For iter  10
Accuracy  0.515625
Loss  0.67319876
__________________
For iter  20
Accuracy  0.6953125
Loss  0.6200237
__________________
For iter  30
Accuracy  0.8984375
Loss  0.25687107
__________________
For iter  40
Accuracy  0.9453125
Loss  0.2739342
__________________
For iter  50
Accuracy  0.890625
Loss  0.27588338
__________________
For iter  60
Accuracy  0.921875
Loss  0.21773568
__________________
For iter  70
Accuracy  0.875
Loss  0.28757942
__________________
For iter  80
Accuracy  0.90625
Loss  0.23174456
__________________
For iter  90
Accuracy  0.8828125
Loss  0.32910788
__________________
For iter  100
Accuracy  0.8984375
Loss  0.2342968
__________________
For iter  110
Accuracy  0.90625
Loss  0.21832019
__________________
For iter  120
Accuracy  0.953125
Loss  0.10605277
___________

In [22]:
sess.run(prediction)

RuntimeError: Attempted to use a closed Session.